<a href="https://colab.research.google.com/github/mykolesiko/eeg_investigation/blob/master/PSD_PLV_version2_2_zscore_5layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

In [ ]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/deap")

In [ ]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 256
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="/content/drive/MyDrive/MADE/Project/RACNN_models/"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()   

In [ ]:
import glob
import pickle
from collections import Counter
import scipy
from  scipy import signal
from scipy.fft import fft, fftfreq

In [ ]:
import glob
import pickle
from sklearn.preprocessing import MinMaxScaler

data = []
labels = []
data_dir = './data_preprocessed_python'
files = glob.glob(os.path.join(data_dir, "*.dat"))
data_raw = []
for file_data in files:
    raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
    data.append(raw_data['data'])
    labels.append(raw_data['labels'])
    #break

In [ ]:
labels_bin = []
for sub in range(32):
#for sub in range(1):
  temp = labels[sub] >= 4.5
  labels_bin.append(temp)
  #print(sum(labels_bin[sub][:, type_emotion]), end=' ')

In [ ]:
print(len(data))

32


In [ ]:
LEN_RECORD_IN_SECONDS = 60
NVIDEOS = 40
HCANALS = 9
WCANALS = 9
NTIMES_IN_SAMPLE = 128
NTIMES_IN_SEC = 128
NCANALS = 32
NFEATURES = 32
NSUBJECTS = 32
electrode_matrix = {}
electrode_matrix['FP1'] = [0, 3]
electrode_matrix['FP2'] = [0, 5]
electrode_matrix['AF3'] = [1, 3]
electrode_matrix['AF4'] = [1, 5]
electrode_matrix['F7']  = [2, 0]
electrode_matrix['F3']  = [2, 2]
electrode_matrix['FZ']  = [2, 4]
electrode_matrix['F4']  = [2, 6]
electrode_matrix['F8']  = [2, 8]
electrode_matrix['FC5']  = [3, 1]
electrode_matrix['FC1']  = [3, 3]
electrode_matrix['FC2']  = [3, 5]
electrode_matrix['FC6']  = [3, 7]
electrode_matrix['T7']  = [4, 0]
electrode_matrix['C3']  = [4, 2]
electrode_matrix['CZ']  = [4, 4]
electrode_matrix['C4']  = [4, 6]
electrode_matrix['T8']  = [4, 8]
electrode_matrix['CP5']  = [5, 1]
electrode_matrix['CP1']  = [5, 3]
electrode_matrix['CP2']  = [5, 5]
electrode_matrix['CP6']  = [5, 7]
electrode_matrix['P7']  = [6, 0]
electrode_matrix['P3']  = [6, 2]
electrode_matrix['PZ']  = [6, 4]
electrode_matrix['P4']  = [6, 6]
electrode_matrix['P8']  = [6, 8]
electrode_matrix['PO3'] = [7, 3]
electrode_matrix['PO4'] = [7, 5]
electrode_matrix['O1'] = [8, 3]
electrode_matrix['OZ'] = [8, 4]
electrode_matrix['O2'] = [8, 5]

list_electrodes = ['FP1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3',	'T7',	'CP5',	'CP1',	'P3',	'P7',	'PO3',	'O1',	'OZ',	'PZ',	'FP2',	'AF4', 'FZ', 'F4', 'F8', 'FC6',	'FC2',	'CZ', 'C4', 'T8', 'CP6',	'CP2',	'P4', 	'P8',	'PO4',	'O2']
data_dir = './data_preprocessed_python'
TRAIN_SIZE = 0.9
THRESHOLD = 4.5

In [ ]:
class EmotionDataset(Dataset):
    def __init__ (self, data, labels_bin, interval, shift, seq, type):  #indexes - индексы видео которые вошли в датасет, data - данные labels - метки бинарные
       self.data_samples = []
       self.labels = []
       self.transforms = transforms
       self.cnt = [Counter(), Counter(), Counter(),Counter()]
       for sub in range(len(data)): #sub   - человек
          for nvideo in range(NVIDEOS):
             for nsec in range(0, LEN_RECORD_IN_SECONDS - interval, shift):
                    #data_sample = data[sub][nvideo, :,  (3 + nsec) * NTIMES_IN_SEC : (3 + nsec + interval) * NTIMES_IN_SEC]
                    #freq, psd_sample = signal.welch(data[0][0], fs=128.0, window='hanning', nperseg=256, noverlap=None, nfft=None, detrend='constant', return_onesided=True, scaling='density', axis=-1)#fft(data_sample)
                    temp = data[sub][nvideo, :,  (3 + nsec) * NTIMES_IN_SEC : (3 + nsec + interval) * NTIMES_IN_SEC].copy()
                    data_sample = np.zeros((NCANALS, NTIMES_IN_SEC + 1))
                    for i in range(NCANALS):
                        freq, data_sample[i] = signal.welch(temp[i], fs=128.0, window='hanning', nperseg=256, noverlap=None, nfft=None, detrend='constant', return_onesided=True, scaling='density', axis=-1)
                    self.data_samples.append(data_sample)
                    self.labels.append(labels_bin[sub][nvideo, :])
       n = len(self.data_samples)
       #seq = np.arange(n)
       #random.shuffle(seq)
       size = int(koeff * n)
       print(n)
       if type == 'train':
            self.data_samples = np.array(self.data_samples)[np.array(seq)][0 : size]
            self.labels = np.array(self.labels)[np.array(seq)][0 : size]
       else:
             self.data_samples = np.array(self.data_samples)[seq][size : ]
             self.labels = np.array(self.labels)[seq][size : ]

    def __len__(self):
       result =  len(self.data_samples)
       return result

    def __getitem__(self, item):
       sample = {}   
       sample = {"labels": self.labels[item],
                 "data": self.data_samples[item]
       }
       #(sub, nvideo, nsec) = sample['data'] 
       #sample['data'] = np.zeros((NCANALS, NTIMES_IN_SEC + 1))
       #temp = data[sub][nvideo, :,  (3 + nsec) * NTIMES_IN_SEC : (3 + nsec + interval) * NTIMES_IN_SEC].copy()
       #print(temp.shape)
       #print(sample['data'].shape)
       #for i in range(NCANALS):
        #    freq, sample['data'][i] = signal.welch(temp[i], fs=128.0, window='hanning', nperseg=256, noverlap=None, nfft=None, detrend='constant', return_onesided=True, scaling='density', axis=-1)

       if transforms is not None:
           for t in transforms:
                sample = t(sample)
       #print(sample)         
       return sample
                 


delta (0-3 Hz), theta (4-7
Hz), low alpha (8-9.5 Hz), high alpha (10.5-12 Hz), alpha (8-
12 Hz), low beta (13-16 Hz), mid beta (17-20 Hz), high beta
(21-29 Hz), beta (13-29 Hz), and gamma (30-50 Hz).

In [ ]:
# class Compose(object):
#     def __init__(self, transforms):
#         super(Compose, self).__init__()
#         self.transforms = transforms

#     def __call__(self, sample):
#         for t in self.transforms:
#             sample = t(sample)
#         return sample
import matplotlib 
from matplotlib import pyplot as plt
 
class to_pcc_matrix_per_freq(object):
    def __init__(self):
        super(to_pcc_matrix_per_freq, self).__init__()
  
    def __call__(self, sample: dict):
         freq_data = sample['data'].copy()
         #plt.plot(freq_data[0])
         freq_ranges_gc = np.array([0, 3, 4, 7, 8, 9.5, 10.5, 12, 8, 12, 13, 16, 17,20, 21, 29, 13, 29, 30, 50])
         koeff = 0.5
         freq_ranges_ind = ((freq_ranges_gc)/0.5).astype(int)
         list_canals = np.arange(NCANALS)
         #random.shuffle(list_canals)
         freq_data_new = freq_data#np.sqrt(np.array(freq_data).real ** 2 + np.array(freq_data).imag ** 2)
         
         freq_data_new = scipy.stats.zscore(freq_data_new)
         #plt.plot(freq_data_new[0])
        #  sum_canals = []
        #  for i in range(NCANALS):
        #      sum_canals.append(freq_data_new[i].sum())
         pcc_matrixes = []
         for s in range(10):
            ind_begin = freq_ranges_ind[s * 2]
            ind_end = freq_ranges_ind[s * 2 + 1]
            #print(ind_begin)
            ##print(ind_end)
            #print(freq_data_new.shape)
            #print("*****")
            #pcc_matrix = np.zeros((9, 9))
            pcc_matrix = np.zeros((27, 27))
            for i in range(NCANALS):
                #print(electrode_matrix[list_electrodes[i]][0])
                #print(freq_data_new)
                pcc_matrix[electrode_matrix[list_electrodes[i]][0] * 3, electrode_matrix[list_electrodes[i]][1] * 3]  = (freq_data_new[i, ind_begin : ind_end ]).sum()#/sum_canals[i]
                
            #np.corrcoef(freq_data_new[:, ind_begin : ind_end])
            # for i in range(NCANALS):
            #    for j in range(i + 1):
            #       pcc_matrix[i, j] = pcc_matrix[i, j]/(np.std(freq_data_new[i, ind_begin : ind_end]) * np.std(freq_data_new[j, ind_begin : ind_end]))
            #       pcc_matrix[j, i] = pcc_matrix[i, j]
            
            pcc_matrixes.append(pcc_matrix)
         #plt.plot(np.array(pcc_matrixes)[:, electrode_matrix[list_electrodes[0]][0] * 3, electrode_matrix[list_electrodes[0]][1] * 3])   
         sample['data'] = np.array(pcc_matrixes)
         #print(pcc_matrix)
         return(sample)




# class to_pcc_matrix(object):
#     def __init__(self):
#         super(to_pcc_matrix, self).__init__()
        

#     def __call__(self, sample: dict):
#         canal_data = sample['data'].copy()
#         list_canals = np.arange(NCANALS)
#         random.shuffle(list_canals)
#         canal_data_new = [canal_data[list_canals[i]] for i in range(32)]
#         pcc_matrix = np.cov(canal_data_new)
#         for i in range(NCANALS):
#            for j in range(i + 1):
#               pcc_matrix[i, j] = pcc_matrix[i, j]/(np.std(canal_data_new[i]) * np.std(canal_data_new[j]))
#               pcc_matrix[j, i] = pcc_matrix[i, j]
#         sample['data'] = pcc_matrix
#         #print(pcc_matrix)
#         return(sample)

class ToTensor(object):

    def __init__(self, ):
        super(ToTensor, self).__init__()

    def __call__(self, sample: dict):
        return {"labels": torch.tensor(sample["labels"], dtype=torch.long),
                "data": torch.tensor(sample["data"], dtype=torch.float32),
                } 


transforms = [to_pcc_matrix_per_freq(),
                   ToTensor()]                       

In [ ]:
def get_conv_module(n_input_channels,n_output_channels):
  conv = nn.Conv2d(n_input_channels, n_output_channels, kernel_size = 3, stride=1, padding='same')
  maxpool = nn.MaxPool2d(kernel_size = 2)
  bn = nn.BatchNorm2d(n_output_channels)
  relu = nn.ReLU()
  result = torch.nn.Sequential(conv, maxpool, bn, relu)
  return(result)

In [ ]:
class EmotionNet(torch.nn.Module): 
   def __init__(self, hcanals, wcanals, tures, ntimes_in_sample):
      super().__init__()
      #print("1")
      n_input_channels = [10, 32, 64, 128]#, 256]
      n_output_channels = [32, 64, 128, 256]#, 512]
      self.convs = nn.ModuleList([get_conv_module(n_input_channels[i], n_output_channels[i]) for i in range(3)])
      self.conv = nn.Conv2d(n_input_channels[3], n_output_channels[3], kernel_size = 3, stride=1, padding='same')
      self.flat = nn.Flatten(1, 3)
      #self.fc = nn.Linear(512, 2)
      #self.fc = nn.Linear(4608, 2)
      self.fc = nn.Linear(2304, 2)

      
   def forward(self, input):
      #print(f"input_shape = {input.shape}")
      #input = input.unsqueeze(3)
      #input = input.permute(0, 3, 1, 2)
      #input (bs, h=32, w=32, 1)

      output = input
      for i, conv in enumerate(self.convs):
            output = conv(output)
            #print(outputs[i])
      output = self.conv(output)
      output = self.flat(output)
      #output (bs, s * 256,  h=9, w=9)
      output = self.fc(output)
      return output



In [ ]:
interval = 3
shift = 1
koeff = 0.8
# #inds = np.arange(2280)
# inds = np.arange(72960)

# #inds = np.arange(12800)
# random.shuffle(inds)
# train_dataset = EmotionDataset(data, labels_bin, interval, shift, inds, 'train')
# train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
#                                pin_memory=True, shuffle=True, drop_last=True)


# val_dataset = EmotionDataset(data, labels_bin, interval, shift, inds, 'test')
# val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
#                                pin_memory=True, shuffle=False, drop_last=False)

In [ ]:
# with open('train_dataset.pkl', 'wb') as fp:
#        pickle.dump(train_dataset, fp)

In [ ]:
# with open('val_dataset.pkl', 'wb') as fp:
#        pickle.dump(val_dataset, fp)

In [ ]:
# with open('train_dataset.pkl', 'rb') as fp:
#        train_dataset = pickle.load(fp)
# with open('val_dataset.pkl', 'rb') as fp:
#         val_dataset = pickle.load(fp)


train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                pin_memory=True, shuffle=True, drop_last=True)


# val_dataset = EmotionDataset(data, labels_bin, interval, shift, inds, 'test')
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                                pin_memory=True, shuffle=False, drop_last=False)        
        

In [ ]:
def train(model, loader, criterion, optimizer, device, batch = None):
    model.train()
    train_loss = []
    inputs = []
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    for batch in tqdm(loader, total=len(loader), desc="training...", position=0 , leave = True):

            optimizer.zero_grad()
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]

            #print(batch)
            #print(trg.shape)
            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            #usual cross entropy
            #output = levels_pred[:, 1:].reshape(-1, levels_pred.shape[-1])
            #trg1 = trg[:, 1:].reshape(-1)
            loss = criterion(levels_pred, trg) 

            #print("after")
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            #break
    return np.mean(train_loss)#, mid_outputs

In [ ]:
def evaluate(model, loader, criterion, device):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        for s, batch in enumerate(tqdm(loader, total=len(loader), desc="validating...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]



            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            
            loss = criterion(levels_pred, trg) 

            epoch_loss += loss.item() 
        
    return epoch_loss / s
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report, f1_score

In [ ]:
def calculate_predictions(model, loader):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        for i, batch in enumerate(tqdm(loader, total=len(loader), desc="predicting...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]
           

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            trg_pred = levels_pred.argmax(1)
            
            real.extend(trg)
            pred.extend(trg_pred) 

            
        print(accuracy_score(real, pred)) 
        print(confusion_matrix(real, pred))  
        print(classification_report(real, pred))   
    f1 = ((f1_score(real, pred, 'binary', pos_label = 0))  + (f1_score(real, pred, 'binary', pos_label = 1)))/2
    return (accuracy_score(real, pred)) , f1        
    

In [ ]:
def get_model():
  model = EmotionNet(HCANALS, WCANALS, NFEATURES, NTIMES_IN_SAMPLE).to(device)
  return model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model()
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

EmotionNet(
  (convs): ModuleList(
    (0): Sequential(
      (0): Conv2d(10, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), paddin

In [ ]:
criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
# print(train_dataset.cnt)
# print(val_dataset.cnt)
# print(files[ind_train])
# print(files[ind_val])

In [ ]:
args.epochs = 20000
#criterion =  fnn.mse_loss
train_loss_min = 10000
f1_min = -10000
#batch = next(iter(train_dataloader))
for epoch in range(args.epochs):
    #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
    
    train_loss = train(model, train_dataloader, criterion, optimizer ,device)
    #if epoch % 500 == 0:
    print(train_loss)

    if (train_loss < train_loss_min):
        train_loss_min      = train_loss
        torch.save({
                         'model_state_dict': model.state_dict(),
                         'optimizer_state_dict': optimizer.state_dict(),
                       },
                       os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", "train_5.tgz")
            )  

    #val_loss = evaluate(model, val_dataloader, criterion, device)
    # #break
    #print(val_loss)
    #break

    acc, f1 = calculate_predictions(model, val_dataloader)
    print(f1, acc)
    if (f1 > f1_min):
         f1_min      =  f1
         torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"val_5.tgz"))

training...:   0%|          | 0/228 [00:00<?, ?it/s]

0.39191688923982154


predicting...:   0%|          | 0/57 [00:00<?, ?it/s]

0.7354714912280702
[[2796 2592]
 [1268 7936]]
              precision    recall  f1-score   support

           0       0.69      0.52      0.59      5388
           1       0.75      0.86      0.80      9204

    accuracy                           0.74     14592
   macro avg       0.72      0.69      0.70     14592
weighted avg       0.73      0.74      0.73     14592

0.697999747612506 0.7354714912280702


training...:   0%|          | 0/228 [00:00<?, ?it/s]

...

0.34825981669781503


predicting...:   0%|          | 0/57 [00:00<?, ?it/s]

0.7445860745614035
[[2923 2465]
 [1262 7942]]
              precision    recall  f1-score   support

           0       0.70      0.54      0.61      5388
           1       0.76      0.86      0.81      9204

    accuracy                           0.74     14592
   macro avg       0.73      0.70      0.71     14592
weighted avg       0.74      0.74      0.74     14592

0.7103147283290524 0.7445860745614035


training...:   0%|          | 0/228 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
print(calculate_predictions(model, val_dataloader))

predicting...:   0%|          | 0/57 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff0aa80ecb0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


0.6895559210526315
[[2205 3214]
 [1316 7857]]
              precision    recall  f1-score   support

           0       0.63      0.41      0.49      5419
           1       0.71      0.86      0.78      9173

    accuracy                           0.69     14592
   macro avg       0.67      0.63      0.63     14592
weighted avg       0.68      0.69      0.67     14592

(0.6895559210526315, 0.6347592923381722)


In [ ]:
val_loss = evaluate(model, val_dataloader, criterion, device)
# #break
print(val_loss)

In [ ]:
model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"val_5.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
model.load_state_dict(model_state['model_state_dict'])
print(calculate_predictions(model, val_dataloader))

predicting...:   0%|          | 0/57 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f19eb5c80e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


0.7451343201754386
[[3232 2156]
 [1563 7641]]
              precision    recall  f1-score   support

           0       0.67      0.60      0.63      5388
           1       0.78      0.83      0.80      9204

    accuracy                           0.75     14592
   macro avg       0.73      0.72      0.72     14592
weighted avg       0.74      0.75      0.74     14592

(0.7451343201754386, 0.7195284609627088)


In [ ]:
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import f1_score, accuracy_score


#def get_mertics_per_subject():
k  = 5
type_emotion = 1

acc_all = []
f1_all = []
for sub in range(0,1):
    print(f"*******{sub}*********")

    args.batch_size = 100
    indexes = np.arange(NVIDEOS * 1 * LEN_RECORD_IN_SECONDS)
    n = len(indexes)
    #X = np.arange(40)
    y = []
    for nvideo in range(NVIDEOS):
        y.extend(60 * [labels_bin[sub][nvideo, type_emotion]])
    

    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(indexes, y)
    acc_sub = []
    f1_sub = []
    for fold,  (inds_train, inds_test) in  enumerate(balanced_split):
        print(f"fold = {fold}")
        #print(inds_train, inds_test)
        #print(sum(labels_bin[sub][inds_train, type_emotion]))
        #print(sum(labels_bin[sub][inds_test, type_emotion]))
        args.batch_size = 100
        #transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
        #transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
        transforms = [to_head_matrix(),ToTensor()] 
        #transforms = [RandomAugmentation([add_noise(), None], 0.5), to_head_matrix(),ToTensor()]   
        #transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   

        train_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[sub : sub + 1], transforms, inds_train)
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                    pin_memory=True, shuffle=True, drop_last=True)

        val_dataset = EmotionDataset(data[sub: sub + 1], labels_bin[sub  : sub + 1], transforms,  inds_test)
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)
        device  = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = get_model(batch_norm = True, reg_extractor = True, ass_extractor = True)
        model.apply(initialize_weights)
        criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
        train_loop(f'batchnorm_Yes_batch_100_Adam_lr=3e4_{sub}_{fold}_subject2', 1, 50)
        description = f'batchnorm_Yes_batch_100_Adam_lr=3e4_{sub}_{fold}_subject2'
        model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        acc, f1 = calculate_predictions(model, val_dataloader, type_emotion)
        print(f"f1 = {f1} acc = {acc}")
        acc_sub.append(acc)
        f1_sub.append(f1)
        print(acc, f1)
    acc_all.append(acc_sub)     
    f1_all.append(f1_sub) 